# NIST SRM 731
### Date:  20190627
### Author:  Donald Windover

## Calculation using the third-order spline provided in the COA to re-create the data tables


### The NIST SRM 731 Certificate of Analysis can be found at:

https://www-s.nist.gov/srmors/certificates/731L1.pdf

#### Python packages loading:

Here we import the numpy, scipy, and scipy.integrate packages used in calculations:

```python
import numpy as np
import scipy as sp
import scipy.integrate as integrate
```

For numpy and scipy documentation, see:

https://docs.scipy.org/doc/



In [1]:
import numpy as np
import scipy as sp
import scipy.integrate as integrate

### Here we impliment the 293<= T <= 640 third-order spline polynomial from the COA

\begin{equation}
\alpha = 0.8651 + 2.3569 \times 10 ^{-2} \dot T - 4.2277 \times 10 ^{-5} \dot T^2 + 2.5408 \times 10^{-8} \dot T^3
\end{equation}

In [2]:
def expansivity_high(Temp):
    expansivity = (0.8651 + (2.3569e-2 * Temp) - (4.2277e-5 * Temp**2) + (2.5408e-8 * Temp**3))
    return expansivity

### We can reproduce the Expansivity values for the table on page 2 from the COA
#### first, by putting in the Temperatures in the table...

In [3]:
Temp_chart = [293, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500, 520, 540, 560, 580, 600, 620, 640, 660, 680]
for i in np.arange(len(Temp_chart)):
    print(Temp_chart[i])

293
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680


### and then, by using our third order polynomial function above to calculate expansivity at each temp, and round to two decimals...

#### Breakdown of the python statement below:

1. the first print statment provides the row information
2. the for statement runs through each temperature
3. the Epansivity is pulled from ln 2 and is calculated for each temperature, rounded to 2 decimal places
4. the temperatue and Expansivity are then printed

In [4]:
print("Temp (K) / Expansivity (1x10-6) ")
for i in np.arange(len(Temp_chart)):
    Expansivity = np.round(expansivity_high(Temp_chart[i]), decimals = 2)
    print(Temp_chart[i], Expansivity)

Temp (K) / Expansivity (1x10-6) 
293 4.78
300 4.82
320 4.91
340 4.99
360 5.06
380 5.11
400 5.15
420 5.19
440 5.21
460 5.23
480 5.25
500 5.26
520 5.26
540 5.27
560 5.27
580 5.27
600 5.27
620 5.28
640 5.29
660 5.31
680 5.33


### We need to integrate the expansivity... Here is my view of intergration


![alt text][integration]

[integration]: https://imgs.xkcd.com/comics/differentiation_and_integration.png "https://imgs.xkcd.com/comics/differentiation_and_integration.png"

https://imgs.xkcd.com/comics/differentiation_and_integration.png

### I chose to perform a simple numerical integration using scipy.
see:  https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html

We are using the relation between Expansivity and Expansion from the COA

\begin{equation}
\alpha \dot dT = {{1} \over {L_{293}}}dL
\end{equation}

\begin{equation}
\int_{293}^T \alpha \dot dT = \int_{L_{293}}^L {{1}\over{L_{293}}} dL = {{L - L_{293}} \over {L_{293}}}
\end{equation}


Breakdown of the python statement below:

1. the first print statment provides the row information
2. the for statement runs through each temperature
3. the Expansivity is numerically integrated between 293 and Temp_chart value from the for loop step to provide expansion
4. the xpansion is rounded to 0 decimal places
5. the Temperature and Expansion are printed

In [5]:
print("Temp (K) / Expansion[Delta L/ L293] (1x10-6) ")
for i in np.arange(len(Temp_chart)):
    Expansion = integrate.quad (lambda x: expansivity_high(x), 293,Temp_chart[i])
    Expansion = np.round(Expansion[0], decimals = 0)
    print(Temp_chart[i], Expansion)

Temp (K) / Expansion[Delta L/ L293] (1x10-6) 
293 0.0
300 34.0
320 131.0
340 230.0
360 330.0
380 432.0
400 535.0
420 638.0
440 742.0
460 847.0
480 952.0
500 1057.0
520 1162.0
540 1267.0
560 1372.0
580 1478.0
600 1583.0
620 1689.0
640 1795.0
660 1901.0
680 2007.0


### But since Expansivity is modeled as a polyomial, we can just do integration on each polynomial term

#### increasing the power by one and dividing by the new power 


\begin{equation}
\int \alpha \dot dT = \int 0.8651 + 2.3569 \times 10 ^{-2} \dot T - 4.2277 \times 10 ^{-5} \dot T^2 + 2.5408 \times 10^{-8} \dot T^3 \dot dT
\end{equation}


\begin{equation}
\int \alpha \dot dT = 0.8651 \dot (T) + 2.3569 \times 10 ^{-2} \dot (T^2/2) - 4.2277 \times 10 ^{-5} \dot (T^3/3) + 2.5408 \times 10^{-8} \dot (T^4/4)
\end{equation}

In [6]:
def expansion_high(Temp):
    expansion = (0.8651* Temp + (2.3569e-2 * Temp**2)/2 - (4.2277e-5 * Temp**3)/3 + (2.5408e-8 * Temp**4)/4)
    return expansion

In [7]:
print("Temp (K) / Expansion[Delta L/ L293] (1x10-6) ")
for i in np.arange(len(Temp_chart)):
    Expansion = expansion_high(Temp_chart[i]) - expansion_high(Temp_chart[0])
    print(Temp_chart[i], np.round(Expansion, decimals = 0))

Temp (K) / Expansion[Delta L/ L293] (1x10-6) 
293 0.0
300 34.0
320 131.0
340 230.0
360 330.0
380 432.0
400 535.0
420 638.0
440 742.0
460 847.0
480 952.0
500 1057.0
520 1162.0
540 1267.0
560 1372.0
580 1478.0
600 1583.0
620 1689.0
640 1795.0
660 1901.0
680 2007.0


### Both methods of intergration check out nicely!

#### we see a slight variaion fo 1ppm at 640 and 660 from tabulated values, but I am guessing we are seeing a rounding error from calculations performed in the 70s.

### Now, lets check the values you needed calculated....

#### We make a new table of temperatures in degrees C, and then convert them to K

````python
Temp_of_interest_in_C = [30, 50, 80, 300]
Temp_of_interest_in_K = Temp_of_interest_in_C + 273.15
````

In [8]:
Temp_of_interest_in_C = [30, 50, 80, 300]
Temp_of_interest_in_K = np.array(Temp_of_interest_in_C) + 273.15
print("Temps: ", Temp_of_interest_in_K)

Temps:  [ 303.15  323.15  353.15  573.15]


#### We then run the expansivity calculations between 300 C and the lower values requested

Breakdown of the python statement below:

1. the first print statment provides the row information
2. the for statement runs through each temperature
3. the Expansivity is called for each temp
4. the Expansivity is rounded to 2 decimal places
5. the Temperature and Expansivity are printed

In [9]:
print("T / Expansivity [T] (1x10-6) ")
for i in np.arange(len(Temp_of_interest_in_K)):
    Expansivity = expansivity_high(Temp_of_interest_in_K[i])
    Expansivity = np.round(Expansivity, decimals = 2)
    print("Expansivity (", Temp_of_interest_in_K[i], ") / ", Expansivity)

T / Expansivity [T] (1x10-6) 
Expansivity ( 303.15 ) /  4.83
Expansivity ( 323.15 ) /  4.92
Expansivity ( 353.15 ) /  5.03
Expansivity ( 573.15 ) /  5.27


#### We then run the expansion calculations between 300 C and the lower values requested

Breakdown of the python statement below:

1. the first print statment provides the row information
2. the for statement runs through each temperature
3. the Expansion is called for each temp, and subtracted from expansion at 300 C
4. the Expansion is rounded to 0 decimal places
5. the Temperature and Expansion are printed

In [10]:
print("Expansion of (Temp (300C)  - Temp (in K)) / Expansion[Delta L_{T}/ L_{573.15}] (1x10-6) ")
for i in np.arange(len(Temp_of_interest_in_K)):
    Expansion = expansion_high(Temp_of_interest_in_K[-1]) - expansion_high(Temp_of_interest_in_K[i])
    Expansion = np.round(Expansion, decimals = 0)
    print("Expansion (", Temp_of_interest_in_K[-1], " - ", Temp_of_interest_in_K[i], ') / ', Expansion)

Expansion of (Temp (300C)  - Temp (in K)) / Expansion[Delta L_{T}/ L_{573.15}] (1x10-6) 
Expansion ( 573.15  -  303.15 ) /  1393.0
Expansion ( 573.15  -  323.15 ) /  1295.0
Expansion ( 573.15  -  353.15 ) /  1146.0
Expansion ( 573.15  -  573.15 ) /  0.0


What was wrong in both of our spreadsheet calculations, is that Expansivity is the expansion value at a given temperature.  You cannot just use the Expansivity at (T) as a constant over the range of temperatures.  Instead, we need to integrate expansivity to get at expansion.  When we did this, the equations matched results from the Expansion and expansivity tables in the COA.